In [2]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import re
import spacy

In [3]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Movie plot text
plot = """
The nation of Panem consists of a wealthy Capitol and twelve poorer districts. As punishment for a past rebellion, each district must provide a boy and girl between the ages of 12 and 18 selected by lottery for the annual Hunger Games. The tributes must fight to the death in an arena; the sole survivor is rewarded with fame and wealth. In her first Reaping, 12-year-old Primrose Everdeen is chosen from District 12. Her older sister Katniss volunteers to take her place. Peeta Mellark, a baker's son who once gave Katniss bread when she was starving, is the other District 12 tribute. Katniss and Peeta are taken to the Capitol, accompanied by their frequently drunk mentor, past victor Haymitch Abernathy. He warns them about the "Career" tributes who train intensively at special academies and almost always win. During a TV interview with Caesar Flickerman, Peeta unexpectedly reveals his love for Katniss. She is outraged, believing it to be a ploy to gain audience support, as "sponsors" may provide in-Games gifts of food, medicine, and tools. However, she discovers Peeta meant what he said. The televised Games begin with half of the tributes killed in the first few minutes; Katniss barely survives ignoring Haymitch's advice to run away from the melee over the tempting supplies and weapons strewn in front of a structure called the Cornucopia. Peeta forms an uneasy alliance with the four Careers. They later find Katniss and corner her up a tree. Rue, hiding in a nearby tree, draws her attention to a poisonous tracker jacker nest hanging from a branch. Katniss drops it on her sleeping besiegers. They all scatter, except for Glimmer, who is killed by the insects. Hallucinating due to tracker jacker venom, Katniss is warned to run away by Peeta. Rue cares for Katniss for a couple of days until she recovers. Meanwhile, the alliance has gathered all the supplies into a pile. Katniss has Rue draw them off, then destroys the stockpile by setting off the mines planted around it. Furious, Cato kills the boy assigned to guard it. As Katniss runs from the scene, she hears Rue calling her name. She finds Rue trapped and releases her. Marvel, a tribute from District 1, throws a spear at Katniss, but she dodges the spear, causing it to stab Rue in the stomach instead. Katniss shoots him dead with an arrow. She then comforts the dying Rue with a song. Afterward, she gathers and arranges flowers around Rue's body. When this is televised, it sparks a riot in Rue's District 11. President Snow summons Seneca Crane, the Gamemaker, to express his displeasure at the way the Games are turning out. Since Katniss and Peeta have been presented to the public as "star-crossed lovers", Haymitch is able to convince Crane to make a rule change to avoid inciting further riots. It is announced that tributes from the same district can win as a pair. Upon hearing this, Katniss searches for Peeta and finds him with an infected sword wound in the leg. She portrays herself as deeply in love with him and gains a sponsor's gift of soup. An announcer proclaims a feast, where the thing each survivor needs most will be provided. Peeta begs her not to risk getting him medicine. Katniss promises not to go, but after he falls asleep, she heads to the feast. Clove ambushes her and pins her down. As Clove gloats, Thresh, the other District 11 tribute, kills Clove after overhearing her tormenting Katniss about killing Rue. He spares Katniss "just this time...for Rue". The medicine works, keeping Peeta mobile. Foxface, the girl from District 5, dies from eating nightlock berries she stole from Peeta; neither knew they are highly poisonous. Crane changes the time of day in the arena to late at night and unleashes a pack of hound-like creatures to speed things up. They kill Thresh and force Katniss and Peeta to flee to the roof of the Cornucopia, where they encounter Cato. After a battle, Katniss wounds Cato with an arrow and Peeta hurls him to the creatures below. Katniss shoots Cato to spare him a prolonged death. With Peeta and Katniss apparently victorious, the rule change allowing two winners is suddenly revoked. Peeta tells Katniss to shoot him. Instead, she gives him half of the nightlock. However, before they can commit suicide, they are hastily proclaimed the victors of the 74th Hunger Games. Haymitch warns Katniss that she has made powerful enemies after her display of defiance. She and Peeta return to District 12, while Crane is locked in a room with a bowl of nightlock berries, and President Snow considers the situation.
"""

# Process the movie plot text with spaCy
doc = nlp(plot)

#make a function that takes in a sentence and returns the subject, a list of adjectifs of the subject, the verb, the object and a list of adjectives of the object
def get_subject_object(sent):
    actions=[]
    for token in sent:
        if token.pos_ == "VERB" :  # Root verb (action)
            action = token.text
            subject = [child.text for child in token.children if child.dep_ == "nsubj" or child.dep_ == "nsubjpass"]
            object_ = [child.text for child in token.children if child.dep_ == "dobj"]
            if subject:
                actions.append((subject[0], action, object_[0] if object_ else None))
                return subject[0], action, object_[0] if object_ else None


for sent in doc.sents:
    print(get_subject_object(sent))


('nation', 'consists', None)
('district', 'provide', 'boy')
('tributes', 'fight', None)
('Everdeen', 'chosen', None)
None
('who', 'gave', 'bread')
('Katniss', 'taken', None)
('He', 'warns', 'them')
('Peeta', 'reveals', 'love')
('sponsors', 'provide', 'gifts')
('she', 'discovers', None)
('Games', 'begin', None)
('Peeta', 'forms', 'alliance')
('They', 'find', 'Katniss')
('Rue', 'draws', 'attention')
('Katniss', 'drops', 'it')
('They', 'scatter', None)
('Katniss', 'warned', None)
('she', 'recovers', None)
('alliance', 'gathered', 'supplies')
('Katniss', 'draw', 'them')
('Cato', 'kills', None)
('Katniss', 'runs', None)
('She', 'finds', None)
('Marvel', 'throws', 'spear')
('Katniss', 'shoots', None)
('She', 'comforts', 'Rue')
('she', 'gathers', None)
('it', 'sparks', 'riot')
('Games', 'turning', None)
('Katniss', 'presented', None)
('It', 'announced', None)
None
('She', 'portrays', 'herself')
('announcer', 'proclaims', 'feast')
('Peeta', 'begs', 'her')
('Katniss', 'promises', None)
('Clove'

In [17]:
# function that given a personn and sentences returns the sentences where the person is cited
def get_sentences_with_person(person, doc):
    sentences = []
    for sent in doc.sents:
        if person in sent.text:
            sentences.append(sent.text)
    return sentences

# test function for every personn in doc.ents whose .label_ is PERSON

#function that returns a list of unique characters
def get_characters(doc):
    characters = []
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            characters.append(ent.text)
    return list(set(characters))


characters = get_characters(doc)
for ent in characters:
    print(ent, get_sentences_with_person(ent, doc))

Glimmer ['They all scatter, except for Glimmer, who is killed by the insects.']
Haymitch Abernathy ['Katniss and Peeta are taken to the Capitol, accompanied by their frequently drunk mentor, past victor Haymitch Abernathy.']
jacker venom ['Hallucinating due to tracker jacker venom, Katniss is warned to run away by Peeta.']
Clove ['Clove ambushes her and pins her down.', 'As Clove gloats, Thresh, the other District 11 tribute, kills Clove after overhearing her tormenting Katniss about killing Rue.']
Haymitch ['Katniss and Peeta are taken to the Capitol, accompanied by their frequently drunk mentor, past victor Haymitch Abernathy.', "The televised Games begin with half of the tributes killed in the first few minutes; Katniss barely survives ignoring Haymitch's advice to run away from the melee over the tempting supplies and weapons strewn in front of a structure called the Cornucopia.", 'Since Katniss and Peeta have been presented to the public as "star-crossed lovers", Haymitch is able 

In [5]:
# Accessing noun chunks
for chunk in doc.noun_chunks:
    print(f"Noun Chunk: {chunk.text}")

Noun Chunk: 
The nation
Noun Chunk: Panem
Noun Chunk: a wealthy Capitol
Noun Chunk: twelve poorer districts
Noun Chunk: punishment
Noun Chunk: a past rebellion
Noun Chunk: each district
Noun Chunk: a boy
Noun Chunk: girl
Noun Chunk: the ages
Noun Chunk: lottery
Noun Chunk: the annual Hunger Games
Noun Chunk: The tributes
Noun Chunk: the death
Noun Chunk: an arena
Noun Chunk: the sole survivor
Noun Chunk: fame
Noun Chunk: wealth
Noun Chunk: her first Reaping
Noun Chunk: 12-year-old Primrose Everdeen
Noun Chunk: District
Noun Chunk: Her older sister
Noun Chunk: Katniss volunteers
Noun Chunk: her place
Noun Chunk: Peeta Mellark
Noun Chunk: a baker's son
Noun Chunk: who
Noun Chunk: Katniss bread
Noun Chunk: she
Noun Chunk: the other District 12 tribute
Noun Chunk: Katniss
Noun Chunk: Peeta
Noun Chunk: the Capitol
Noun Chunk: their frequently drunk mentor
Noun Chunk: victor Haymitch Abernathy
Noun Chunk: He
Noun Chunk: them
Noun Chunk: the "Career" tributes
Noun Chunk: who
Noun Chunk: speci

In [4]:
for token in doc:
    print(token.text, token.lemma_)


 

The the
nation nation
of of
Panem Panem
consists consist
of of
a a
wealthy wealthy
Capitol Capitol
and and
twelve twelve
poorer poor
districts district
. .
As as
punishment punishment
for for
a a
past past
rebellion rebellion
, ,
each each
district district
must must
provide provide
a a
boy boy
and and
girl girl
between between
the the
ages age
of of
12 12
and and
18 18
selected select
by by
lottery lottery
for for
the the
annual annual
Hunger Hunger
Games Games
. .
The the
tributes tribute
must must
fight fight
to to
the the
death death
in in
an an
arena arena
; ;
the the
sole sole
survivor survivor
is be
rewarded reward
with with
fame fame
and and
wealth wealth
. .
In in
her -PRON-
first first
Reaping Reaping
, ,
12-year 12-year
- -
old old
Primrose Primrose
Everdeen Everdeen
is be
chosen choose
from from
District District
12 12
. .
Her -PRON-
older old
sister sister
Katniss Katniss
volunteers volunteer
to to
take take
her -PRON-
place place
. .
Peeta Peeta
Mellark Mellark
, ,
a 

In [3]:
for sentence in doc.sents:
    print(sentence)


The nation of Panem consists of a wealthy Capitol and twelve poorer districts.
As punishment for a past rebellion, each district must provide a boy and girl between the ages of 12 and 18 selected by lottery for the annual Hunger Games.
The tributes must fight to the death in an arena; the sole survivor is rewarded with fame and wealth.
In her first Reaping, 12-year-old Primrose Everdeen is chosen from District 12.
Her older sister Katniss volunteers to take her place.
Peeta Mellark, a baker's son who once gave Katniss bread when she was starving, is the other District 12 tribute.
Katniss and Peeta are taken to the Capitol, accompanied by their frequently drunk mentor, past victor Haymitch Abernathy.
He warns them about the "Career" tributes who train intensively at special academies and almost always win.
During a TV interview with Caesar Flickerman, Peeta unexpectedly reveals his love for Katniss.
She is outraged, believing it to be a ploy to gain audience support, as "sponsors" may 

In [28]:
def get_subject_object(doc):
    interactions=[]
    for token in doc:
        if token.pos_=="VERB":
            subj=[child.text for child in token.children if child.dep_ == "nsubj" or child.dep_ == "nsubjpass"]
            subj=subj[0] if subj else ""
            obj=[child.text for child in token.children if child.dep_ == "dobj" or child.dep_ == "pobj"]
            obj=obj[0] if obj else ""
            if subj in characters and obj in characters:
                interactions.append([subj, token.lemma_, obj])
    return interactions

interactions = get_subject_object(doc)
interactions

[['Peeta', 'tell', 'Katniss'], ['Haymitch', 'warn', 'Katniss']]

In [24]:
labels=set()
for ent in doc.ents:
    labels.add(ent.label_)
print(labels)

{'TIME', 'CARDINAL', 'FAC', 'ORG', 'DATE', 'LOC', 'GPE', 'EVENT', 'ORDINAL', 'WORK_OF_ART', 'PERSON', 'PRODUCT'}


In [25]:
pos=set()
for token in doc:
    pos.add(token.pos_)
print(pos)

{'SPACE', 'NUM', 'AUX', 'ADV', 'NOUN', 'DET', 'PROPN', 'PUNCT', 'CCONJ', 'PRON', 'ADP', 'VERB', 'ADJ', 'PART', 'SCONJ'}


In [39]:
dep=set()
for token in doc:
    dep.add(token.dep_)
print(dep)

{'conj', 'nmod', 'case', 'nsubj', 'aux', 'predet', 'prt', 'acomp', 'preconj', 'pobj', 'prep', 'dep', 'advmod', 'attr', 'punct', 'auxpass', 'xcomp', 'amod', 'advcl', 'neg', 'pcomp', 'det', 'ROOT', 'agent', 'dative', 'appos', 'relcl', 'mark', 'nsubjpass', 'oprd', 'poss', 'cc', 'dobj', 'npadvmod', 'nummod', 'compound', 'ccomp', 'acl'}


In [29]:
#her PRON poss
for token in list(doc)[60:]:
    if token.pos_=="PRON":
        print(token.text, token.head, [child for child in token.children])

who gave []
she starving []
He warns []
them warns []
who train []
She is []
it be []
she discovers []
what said []
he said []
They find []
her corner []
it drops []
They scatter [all]
who killed []
she recovers []
them draw []
it around []
it guard []
she hears []
She finds []
her releases []
she dodges []
it stab []
him dead []
She comforts []
she gathers []
it sparks []
It announced []
him finds []
She portrays []
herself portrays []
him with []
her begs []
him getting []
he falls []
she heads []
her ambushes []
her pins []
He spares []
she stole []
they are []
They kill []
they encounter []
him hurls []
him spare []
him shoot []
she gives []
him gives []
they commit []
they proclaimed []
she made []
She return [and, Peeta]


In [46]:
for token in doc.ents:
    print(token.text, token.label_)

Panem ORG
Capitol ORG
twelve CARDINAL
between the ages of 12 and 18 DATE
annual DATE
Hunger Games EVENT
12-year-old DATE
Primrose Everdeen ORG
District 12 FAC
Katniss PERSON
Peeta Mellark PERSON
baker PERSON
Katniss ORG
District 12 DATE
Katniss PERSON
Peeta PERSON
Capitol FAC
Haymitch Abernathy PERSON
Career WORK_OF_ART
Caesar Flickerman ORG
Peeta ORG
Katniss PERSON
Peeta PERSON
half CARDINAL
the first few minutes TIME
Katniss ORG
Haymitch PERSON
Cornucopia GPE
four CARDINAL
Katniss ORG
Katniss ORG
Glimmer ORG
Katniss ORG
Peeta ORG
Katniss ORG
a couple of days DATE
Katniss ORG
Cato ORG
Katniss ORG
Rue PERSON
District 1 DATE
Katniss ORG
Rue PERSON
Rue WORK_OF_ART
Rue ORG
Rue's FAC
District 11 DATE
Snow PERSON
Seneca Crane PERSON
Katniss PERSON
Peeta PERSON
Haymitch PERSON
Katniss ORG
Peeta PRODUCT
Katniss ORG
Thresh GPE
District 11 LOC
Katniss ORG
Rue WORK_OF_ART
Rue WORK_OF_ART
Peeta ORG
Foxface ORG
District 5 DATE
Peeta ORG
late at night TIME
Thresh PERSON
Cornucopia GPE
Cato PERSON
K